In [24]:
from nemoguardrails import LLMRails, RailsConfig

from dotenv import load_dotenv
import os 

import openai
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model="gpt-4o")

In [25]:
# API Configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [26]:
import nest_asyncio
nest_asyncio.apply()

# Load NeMo Guardrails configuration
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

In [27]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import START, MessagesState, StateGraph
from psycopg import Connection

In [28]:
# Database connection details
DB_URI = os.getenv("DB_URI")

connection_kwargs = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432",
    "autocommit": True,
    "prepare_threshold": 0,
}

In [60]:
# wrapper function for NeMo execution with persistent state management

def call_nemo(state: MessagesState):

    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )

    # incorporate only the latest 5 messages + first message
    trimmed_messages = state['messages'][0] + state['messages'][-5:]

    print([SystemMessage(content=system_prompt)])
    print()
    print(state['messages'])

    # construct prompt
    prompt = [SystemMessage(content=system_prompt)] + state['messages']

    print(prompt)

    # get response
    res = model.invoke(prompt)

    return {"messages": res}

    # # apply NeMo
    # check_res = rails.generate(res)

    # if "Yes" in check_res:

    #     print("***in check res***")

    #     print(res)
    #     print()
    #     print(check_res)
    #     print()
    #     # store the first and last 5 interactions
    #     # new_state = {"messages": trimmed_messages + [("ai", check_res)]}

    #     # return new_state
    
    # return {"messages": res}

In [61]:
# Define LangGraph workflow with persistence and guardrails
workflow = StateGraph(state_schema=MessagesState)

# Add the node and edge to the workflow
workflow.add_node("model", call_nemo)
workflow.add_edge(START, "model")

In [62]:
# open database connection + test

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # Compile workflow with persistence
    app = workflow.compile(checkpointer=checkpointer)

    # Example configuration and input
    config = {"configurable": {"thread_id": "3"}}
    input_data = {"messages": [("human", "what's the weather in Denver?")]}

    # Run the workflow
    response = app.invoke(input_data, config)

    # Retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)


[SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability.', additional_kwargs={}, response_metadata={})]

[HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='425b5a27-dd69-4310-827b-a458c85cbd41'), HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='f7bfa7c2-bd49-4e59-9381-28e05dc87a75'), HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='e561e7f8-913a-482d-8d82-d9169246de1f'), HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='aa25e443-8c52-4b20-a0ef-3df47199e5ba'), HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ffdcec10-60a8-4ff0-ac72-70f8efee1a32'), HumanMessage(content="what's the weather in Denver?", additional_kwargs={}, response_metadata={}, id='ddf39dee-b9bc-4ed7-9391-b6a456d179d8

TypeError: sequence item 0: expected str instance, AIMessage found